# 1. Setup

## 1.1. Installing and importing libraries to the collab new session (therefore new environment)

In [1]:
!python -m nltk.downloader 'punkt'
#restart the runtime once installed

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install "tqdm==4.43.0"

     |████████████████████████████████| 59 kB 3.0 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [3]:
!pip install sumy

     |████████████████████████████████| 83 kB 1.6 MB/s 
     |████████████████████████████████| 10.1 MB 32.8 MB/s 
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21711 sha256=edc9d6ddf66cb25e8eaf2ae0e6075b7fee6585c39337587d0cee1277c0554c3a
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746883 sha256=8113eb4ff6b05126bb3094e798c145ebe2d70b7c62e0954bd823ff5d3ef7d806
  Stored in directory: /root/.cache/pip/wheels/57/e8/3f/120ccc1ff7541c108bc5d656e2a14c39da0d824653b62284c6
Successfully built breadability pycountry


In [4]:
!pip install datasets

     |████████████████████████████████| 542 kB 5.2 MB/s 
     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 243 kB 36.9 MB/s 
     |████████████████████████████████| 118 kB 64.5 MB/s 


In [5]:
!pip install rouge_score

In [6]:
# Importing necessary librairies
import datasets
from datasets import load_metric
import nltk
import numpy as np
import pandas as pd
import pickle 
from sklearn.model_selection import train_test_split
import spacy
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
import tqdm
from google.colab import drive

## 1.2. Importing personal drive to file in order to load the data


In [7]:
# Mount Google Drive to this Notebook instance.
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Data Formatting

## 2.1. Loading clean data 

In [8]:
# We load the cleaned data from a pickle format
file ='/content/drive/MyDrive/Colab Notebooks/Data/final_clean.pickle'
df = pd.read_pickle(file)
# Looking at the data
df

,description_narrative,TLDR,text_clean,text_embedding,summary_clean,summary_embedding,labels,labels_idx_list,doc_label
0,The training improved women’s knowledge on the...,The training improved women’s knowledge on the...,[The training improved women’s knowledge on th...,"[[0.76747984, -0.1894493, 0.51285785, -0.02116...",[The training improved women’s knowledge on th...,"[[0.76747984, -0.1894493, 0.51285785, -0.02116...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[0],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Illegal oil refining in the Niger Delta is inc...,CEHRD in an effort to create awareness on the ...,[Illegal oil refining in the Niger Delta is in...,"[[-0.26888534, -0.6092957, -0.23817927, -1.167...",[CEHRD in an effort to create awareness on the...,"[[-0.43648192, -0.071132354, -0.18081759, -0.4...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 3]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,CEHRD successfully set-up 4 formal and 4 infor...,CEHRD set-up the environmental clubs with the ...,[CEHRD successfully set-up 4 formal and 4 info...,"[[-0.48964322, -1.2085572, 1.0374501, 0.021369...",[CEHRD set-up the environmental clubs with the...,"[[-0.15335692, -0.29430717, 0.58692193, -1.126...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[10],"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
3,PROJECT NAME : Delivering Accelerated Family P...,Delivering Accelerated Family Planning in Paki...,"[PROJECT NAME :, Delivering Accelerated Family...","[[-0.23698506, 0.15983863, -0.071198896, -1.20...",[Delivering Accelerated Family Planning in Pak...,"[[-0.50266284, -1.2923895, 0.4206834, -1.25673...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[1],"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
4,Road traffic injuries are world's eighth leadi...,The Cardiff Trauma Pack Research and Develop...,[Road traffic injuries are world's eighth lead...,"[[-0.4666822, -1.19177, 0.9945381, -0.95759714...",[ The Cardiff Trauma Pack Research and Develo...,"[[-0.4679708, -0.43502182, 0.8388376, -1.47547...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[4, 5]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
...,...,...,...,...,...,...,...,...,...
2980,Metta has been responding to the humanitarian ...,"ProjectGoal: To improve the condition of 2,854...",[Metta has been responding to the humanitarian...,"[[-0.6875755, -1.0828758, 0.32236806, -1.33987...","[ProjectGoal:, To improve the condition of 2,8...","[[-0.58209735, 0.027911462, 0.29285246, -1.098...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[7, 8]","[2980, 2980, 2980, 2980, 2980, 2980, 2980, 298..."
2981,"Destined Women is local not for profit, non-re...",GOAL: Contribute towards changing the socioeco...,"[Destined Women is local not for profit, non-r...","[[-0.3787009, -0.7928037, 0.16012278, -0.16681...","[GOAL:, Contribute towards changing the socioe...","[[-0.2746246, 0.04018628, 0.4832729, -0.996766...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1, 7]","[2981, 2981, 2981, 2981, 2981, 2981, 2981, 298..."
2982,The project will empower secondary school stud...,The project will empower secondary school stud...,[The project will empower secondary school stu...,"[[0.3152278, -0.62204534, 1.0845532, -0.392865...",[The project will empower secondary school stu...,"[[0.3152278, -0.62204534, 1.0845532, -0.392865...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0],"[2982, 2982, 2982, 2982, 2982, 2982, 2982, 2982]"
2983,The project has been specifically designed to ...,The project has been specifically designed to ...,[The project has been specifically designed to...,"[[0.2448853, -0.043037124, 0.5578783, -0.51006...",[The project has been specifically designed to...,"[[0.2448853, -0.043037124, 0.5578783, -0.51006...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0],"[2983, 2983, 2983, 2983, 2983, 2983]"


In [9]:
# Renaming the columns and
df = pd.DataFrame({'document': df['description_narrative'],
                   'summary': df['TLDR'],
                   'label' :  df['labels']
                   })

## 2.2. Splitting in training, validation and test datasets

In [10]:
# Using skicit learn library, we split the data, keeping 80% for training, 10% for validation and 10% for test data
# we note that in this notebook we will only use the test dataset as this is a baseline model 
train_dataset, test_dataset = train_test_split(df, test_size=0.2)
validation_dataset, test_dataset = train_test_split(test_dataset, test_size=0.5)

# 3. Loading the summariser 

In [11]:
# Defining a function to parse, tokenize and summarise the text 
def summariser(doc):
  parser = PlaintextParser.from_string(doc, Tokenizer('english'))
  lexrank = LexRankSummarizer()
  summary = lexrank(parser.document, 2)
  return summary

# Applying the function to the dataset
#creating an empty list
lex_rank_summaries = []

#looping in the texts of teh datasets
for doc in test_dataset['document']:
  #applying the function to each text 
  #appending the result to the list 
  lex_rank_summaries.append(summariser(doc))

# Creating a column filled with the generated summaries 
test_dataset['predicted_summaries'] = lex_rank_summaries
test_dataset = test_dataset.reset_index()
test_dataset

,index,document,summary,label,predicted_summaries
0,2736,Background on CSBAG: The Civil Society Budget ...,The overall SP-F4D project goal: is to support...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",(Background on CSBAG: The Civil Society Budget...
1,1569,Uganda - TB/HIV - The AIDS Support Organisatio...,This program has the following goals: To reduc...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(To strengthen TB-HIV integrated care for co-i...
2,139,MCC Funding will be used to construct a small ...,This project aims to construct a small run-of-...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",(MCC Funding will be used to construct a small...
3,2127,The Building Afghan Peace Locally programme is...,Objective: to strengthen civil society capacit...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(The total amount of the programme( 958,539) ..."
4,1625,The development objective of the Guinea-Bissau...,The development objective of the Guinea-Bissau...,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(The development objective of the Guinea-Bissa...
...,...,...,...,...,...
282,2718,This project aims to encourage collaboration b...,This project aims to encourage collaboration b...,"[1.0, 0.0, 0.0, 0.0]",(This project aims to encourage collaboration ...
283,2527,Women and Girls Advancement and Resource Centr...,Summary:nnVoice Innovation lab aims at establi...,"[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",(Women and Girls Advancement and Resource Cent...
284,2203,MIFTAH is one of four counterparts of the proj...,"Within AMAL project, MIFTAH continues targetin...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]",(MIFTAHs role within those communities is to o...
285,2953,The latest findings from a countrywide seasona...,xfam provides integrated EFSVL (Emergency Food...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",(The latest findings from a countrywide season...


# 4. Evaluation

## 4.1. Data Preparation

In [12]:
# Creating lists of the reference and the predicted summaries
reference_summaries = test_dataset['summary']
predicted_summaries = test_dataset['predicted_summaries']

# Converting the predicted summaries to strings 
predicted_summaries = [str(sent) for sent in predicted_summaries]

# Defining a function to clean the predicted sentences from the added format generated by text_rank
def cleaner(sent):
  sent = sent.replace("(<Sentence: ", "")
  sent = sent.replace(" <Sentence:", "")
  sent = sent.replace ('>)', "")
  sent = sent.replace('>,', "")
  return sent 

# Applying the function to both the predicted and the reference summaries for consistency
predicted_summaries = [cleaner(sent) for sent in predicted_summaries]
reference_summaries = [cleaner(sent) for sent in reference_summaries]

## 4.2. Computing the ROUGE scores

In [13]:
# Loading the ROUGE metric from Hugginface 
metric = load_metric('rouge')
rouge_scores = metric.compute(predictions=predicted_summaries, references=reference_summaries)
rouge_scores

{'rouge1': AggregateScore(low=Score(precision=0.32460630708701094, recall=0.6891633356921039, fmeasure=0.4173353899679778), mid=Score(precision=0.3484964117468597, recall=0.7247310368279778, fmeasure=0.4442553277559502), high=Score(precision=0.3736331345737549, recall=0.7574098891577571, fmeasure=0.47241229267221896)),
 'rouge2': AggregateScore(low=Score(precision=0.24268142993306743, recall=0.5284749566525734, fmeasure=0.3153122197094201), mid=Score(precision=0.27309325108499694, recall=0.5837664316496936, fmeasure=0.35172753473679225), high=Score(precision=0.3054895813795042, recall=0.6338828811886785, fmeasure=0.38976476718156944)),
 'rougeL': AggregateScore(low=Score(precision=0.288386080723664, recall=0.6212533822121322, fmeasure=0.37291437686209206), mid=Score(precision=0.31682922617851605, recall=0.661376515981331, fmeasure=0.40487695272520896), high=Score(precision=0.3456445913795044, recall=0.7023039844646459, fmeasure=0.4367227108005091)),
 'rougeLsum': AggregateScore(low=Sco

## 4.3. View a few examples

In [14]:
print('This is the reference summary: ')
print(reference_summaries[16])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[16])

This is the reference summary: 
 By building knowledge, skills and expertise at local to global levels this project aims to scale up household methodologiesto empower millions of women, men and rural youth
   
This is the predicted summary: 
The goal of this project is to scale up the empowerment of marginalized rural women and men through the use of Household Methodologies (HHM). A network of local/national learning centres provide opportunities for exposure to the methodology and its benefits and gaining the basic and advanced skills to start the process in the countries.n2.


In [15]:
print('This is the reference summary: ')
print(reference_summaries[69])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[69])

This is the reference summary: 
In light of the context and gapnanalysis, the proposed intervention seeks the support of the UN PeacebuildingnFund to increase the inclusion of young men and women in state andnpeacebuilding processes in Somalia. 
   
This is the predicted summary: 
While otherndonor funds are supporting stabilization and peacebuilding efforts at the communitynlevel, as well as the development of local government institutions and systems,nthere is currently no intervention specifically promoting the inclusion ofnyoung men in women in state and peacebuilding processes in South West State andnJubbaland. After assessing the needs of young women’s organizations,nthe programme decided to employ a gender mixed approach to the differentninitiatives, bringing young women and men together, enabling a collaborativenapproach, while sensitizing young men in the process.The programme will also providentraining and strengthen the institutional capacity of the local administrationsnin 

In [16]:
print('This is the reference summary: ')
print(reference_summaries[103])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[103])

This is the reference summary: 
The standalone Development Policy Loan (DPL) in the amount of US dollar 400 million supports the Government of Colombia's (GoC's) efforts to strengthen institutions for territorial development.
   
This is the predicted summary: 
The standalone Development Policy Loan (DPL) in the amount of US dollar 400 million supports the Government of Colombia's (GoC's) efforts to strengthen institutions for territorial development. Building on previous efforts to address development challenges linked to disparities at the territorial level, stagnating productivity and inefficiencies in public expenditures, the GoC has embarked upon the next phase of implementation of structural reforms to strengthen territorial development focusing on measures to address institutional constraints.
